In [1]:
import ee
%matplotlib inline
import math
import warnings
import intake
warnings.filterwarnings('ignore')
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import cftime
import xarray as xr
xr.set_options(display_style='html')

import datetime, calendar

plt.style.use("seaborn-darkgrid")

#ee.Authenticate()
ee.Initialize()

In [2]:
HIST_START = 1980
HIST_END = 2014

PERCENTILE_STARTYEAR = 1980
PERCENTILE_ENDYEAR = 2019

NUM_BEST_MODELS = 3

In [3]:
FUTURE_YEARS = [(2080, 2099)]
FUTURE_SCENARIOS = ['ssp119', 'ssp126']

In [45]:
with open('citiesonly.csv', 'r', encoding='utf-8') as ifile:
    CITYLATLON = {i[0]: (float(i[1]), float(i[2]), i[3]) for i in [j.split(',') + [idx] for idx, j in enumerate(ifile.readlines()[1:])]}

In [5]:
MODELS = {
    'tasmax': ('GFDL-ESM4', 'CanESM5', 'MRI-ESM2-0', 'IPSL-CM6A-LR', 'EC-Earth3-Veg-LR'),
    'tasmin': ('GFDL-ESM4', 'IPSL-CM6A-LR', 'CanESM5', 'MRI-ESM2-0', 'EC-Earth3-Veg-LR'),
    'pr': ('GFDL-ESM4', 'IPSL-CM6A-LR', 'CanESM5', 'MRI-ESM2-0', 'EC-Earth3-Veg-LR'),
    'hurs': ('GFDL-ESM4', 'CanESM5', 'MRI-ESM2-0', 'IPSL-CM6A-LR', 'EC-Earth3-Veg-LR')
}

MODELRUN = 'r1i1p1f1'

MODELGRID = {
    'GFDL-ESM4': 'gr1',
    'CanESM5': 'gn',
    'MRI-ESM2-0': 'gn',
    'IPSL-CM6A-LR': 'gr',
    'EC-Earth3-Veg-LR': 'gr'
}

YEARLENGTH = {
    'GFDL-ESM4': 365,
    'CanESM5': 365,
    'MRI-ESM2-0': 366,
    'IPSL-CM6A-LR': 366,
    'EC-Earth3-Veg-LR': 366
}

In [6]:
MODEL_FAMILY = {'UKESM1-0-LL': 'HadAM',
 'NorESM2-MM': 'CCM',
 'NorESM2-LM': 'CCM',
 'MRI-ESM2-0': 'UCLA GCM',
 'MPI-ESM1-2-LR': 'ECMWF',
 'MPI-ESM1-2-HR': 'ECMWF',
 'MIROC6': 'MIROC',
 'MIROC-ES2L': 'MIROC',
 'KIOST-ESM': 'GFDL',
 'KACE-1-0-G': 'HadAM',
 'IPSL-CM6A-LR': 'IPSL',
 'INM-CM5-0': 'INM',
 'INM-CM4-8': 'INM',
 'HadGEM3-GC31-MM': 'HadAM',
 'HadGEM3-GC31-LL': 'HadAM',
 'GFDL-ESM4': 'GFDL',
 'GFDL-CM4_gr2': 'GFDL',
 'GFDL-CM4': 'GFDL',
 'FGOALS-g3': 'CCM',
 'EC-Earth3-Veg-LR': 'ECMWF',
 'EC-Earth3': 'ECMWF',
 'CanESM5': 'CanAM',
 'CNRM-ESM2-1': 'ECMWF',
 'CNRM-CM6-1': 'ECMWF',
 'CMCC-ESM2': 'CCM',
 'CMCC-CM2-SR5': 'CCM',
 'BCC-CSM2-MR': 'CCM',
 'ACCESS-ESM1-5': 'HadAM',
 'ACCESS-CM2': 'HadAM',
 'TaiESM1': 'CCM',
}

In [7]:
VARIABLES = {
    'tasmax': {
        'era_varname': 'maximum_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },}
temp = {
    'tasmin': {
        'era_varname': 'minimum_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },
    'pr': {
        'era_varname': 'total_precipitation',
        'nex_transform': lambda x: x * 86400,
        'era_transform': lambda x: x * 1000
    }   
}

In [8]:
CAT_URL = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
COLL = intake.open_esm_datastore(CAT_URL)

class Dataset:
    def __init__(self, varname, model, scenario):
        self.varname =varname
        self.model = model
        self.scenario = scenario
        
        print('Extracting {0} {1} {2}'.format(varname, scenario, model))
        
        run = MODELRUN
        grid = MODELGRID[model]
        
        cat = COLL.search(experiment_id=scenario, table_id='day', variable_id=varname, source_id=model, member_id=run, grid_label=grid)
        ds_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, progressbar=False)
        ds = list(ds_dict.values())[0][varname][0][0].sel(time=slice(['{0}-07-01'.format(min([i[0]-1 for i in FUTURE_YEARS])), '{0}-01-01'.format(HIST_START)][int(scenario=='historical')], ['{0}-12-31'.format(max([i[1] for i in FUTURE_YEARS])), '{0}-12-31'.format(HIST_END)][int(scenario=='historical')])).sel(lat=[i[0] for i in CITYLATLON.values()], lon=[[i[1], i[1]+360][int(i[1]<0)] for i in CITYLATLON.values()], method='nearest')
        #ds = ds.select(lat=[i[0] for i in CITYLATLON.values()], lon=[[i[1], i[1]+360][int(i[1]<0)] for i in citylatlon.values()], method='nearest')
        self.data = ds.to_numpy()

        self.times = [str(d)[:10] for d in ds.time.data]
        
        del(ds)
        
    def get_timeseries(self, dates, loc_id):
        timestart_idx = self.times.index(dates[0])
        timeend_idx = self.times.index(dates[1]) + 1
        return self.data[timestart_idx:timeend_idx, loc_id, loc_id]

In [70]:
def calendardate_percentiles(nex_varname, q, latlon, sh_hem=False):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(PERCENTILE_STARTYEAR, PERCENTILE_ENDYEAR):
        allyears.append(VARIABLES[nex_varname]['era_transform'](get_eravar(era_varname, latlon, start_year=year, end_year=year, southern_hem=False)))
    if not sh_hem:
        return np.percentile(np.vstack(allyears), q, axis=0)
    else:
        res = np.percentile(np.vstack(allyears), q, axis=0)
        return np.concatenate([res[152:], res[:152]])

def wholeyear_percentile(nex_varname, q, latlon):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append(VARIABLES[nex_varname]['era_transform'](get_eravar(era_varname, latlon, start_year=year, end_year=year, southern_hem=False)))
    return np.percentile(np.concatenate(allyears).flatten(), q)

def yearextreme_percentile(nex_varname, q, latlon, wantmax):
    era_varname = VARIABLES[nex_varname]['era_varname']
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append([np.min, np.max][int(wantmax)](VARIABLES[nex_varname]['era_transform'](get_eravar(era_varname, latlon, start_year=year, end_year=year, southern_hem=False))))
    return np.percentile(np.array(allyears), q)

def thresholdexceedance_mediancount(nex_varname, threshold, latlon, want_gte):
    era_varname = VARIABLES[nex_varname]['era_varname']
    data = VARIABLES[nex_varname]['era_transform'](get_eravar(era_varname, latlon, start_year=PERCENTILE_STARTYEAR, end_year=PERCENTILE_ENDYEAR, southern_hem=False))
    if data.size % 365 != 0:
        raise Exception('Data array length is not an integer multiple of 365')
    byyear = data.reshape(data.size//365, 365)
    if want_gte:
        return np.median(np.sum(byyear >= threshold, axis=1))
    else:
        return np.median(np.sum(byyear <= threshold, axis=1))


def get_rmsd(d1, d2):
    c1 = seasonal_means(d1)
    c2 = seasonal_means(d2)
    return np.sqrt(np.mean(np.sum((c1 - c2)**2)))

def count_runs(tf_array, min_runsize):
    falses = np.zeros(tf_array.shape[0]).reshape((tf_array.shape[0],1))
    extended_a = np.concatenate([[0], tf_array, [0]])
    df = np.diff(extended_a)
    starts = np.nonzero(df == 1)[0]
    ends = np.nonzero(df == -1)[0]
    count = 0
    for idx in range(starts.size):
        if ends[idx] - starts[idx] >= min_runsize:
            count += 1
    return count

def removeLeapDays(arr, start_year, end_year, extralong=False, southern_hem=False):
    if extralong:
        indices = list(range(184))
        jan1_idx = 184

        for year in range(start_year, end_year+1):
            indices += [jan1_idx + i for i in range(365)]
            jan1_idx += 365
            if calendar.isleap(year):
                jan1_idx += 1
        return arr[indices]
    elif not southern_hem:
        indices = []
        jan1_idx = 0
        for year in range(start_year, end_year+1):
            indices += [jan1_idx + i for i in range(365)]
            jan1_idx += 365
            if calendar.isleap(year):
                jan1_idx += 1
        return arr[indices]
    else:
        indices = []
        jul1_idx = 0
        for year in range(start_year-1, end_year):
            indices += [jul1_idx + i for i in range(365)]
            jul1_idx += 365
            if calendar.isleap(year):
                jul1_idx += 1
        return arr[indices]

def get_eravar(varname, latlon, start_year, end_year, southern_hem=False, extralong=False):
    model = 'ERA5'
    dataset = ee.ImageCollection("ECMWF/ERA5/DAILY")
    gee_geom = ee.Geometry.Point((latlon[1], latlon[0]))
    if extralong:
        data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-07-01'.format(start_year-1), '{0}-01-01'.format(end_year+1)))
    elif not southern_hem:
        data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-01-01'.format(start_year), '{0}-01-01'.format(end_year+1)))
    else:
        data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-07-01'.format(start_year-1), '{0}-07-01'.format(end_year)))
    result = [i[4] for i in data_vars.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
    return removeLeapDays(np.array(result), start_year, end_year, extralong=extralong, southern_hem=southern_hem)

def get_var(varname, model, loc_id, start_year, end_year, southern_hem=False, extralong=False, scenario='ssp585'):
    scenario = [scenario, 'historical'][int(start_year < 2015)]
    dataset = datasets[(varname, model, scenario)]
    if extralong:
        dates = ('{0}-07-01'.format(start_year-1), '{0}-12-31'.format(end_year))
    elif not southern_hem:
        dates = ('{0}-01-01'.format(start_year), '{0}-12-31'.format(end_year))
    else:
        dates = ('{0}-07-01'.format(start_year-1), '{0}-06-30'.format(end_year))
    
    ds = dataset.get_timeseries(dates, loc_id)
    if YEARLENGTH[model] == 366:
        return removeLeapDays(np.array(ds), start_year, end_year, extralong=extralong, southern_hem=southern_hem)
    else:
        return np.array(ds)

    
def quarters(d, start_year, end_year, southern_hem=False):
    q2 = []  # 60-151
    q3 = []  # 152-243
    q4 = []  # 244-334
    q1 = []  # 335-59
    if not southern_hem:
        jan1_idx = 365
        for year in range(start_year, end_year):
            tmp = np.concatenate((d[jan1_idx - 365 : jan1_idx - 365 + 60], d[jan1_idx + 335 : jan1_idx + 365]), axis=0)
            q1.append(tmp)
            q2.append(d[jan1_idx + 60 : jan1_idx + 152])
            q3.append(d[jan1_idx + 152 : jan1_idx + 244])
            q4.append(d[jan1_idx + 244 : jan1_idx + 335])

            jan1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
        mam_res = np.vstack(q2)
        jja_res = np.vstack(q3)
        son_res = np.vstack(q4)
        djf_res = np.vstack(q1)
    else:
        jul1_idx = 365
        for year in range(start_year, end_year+1):
            tmp = np.concatenate((d[jul1_idx - 365 : jul1_idx - 365 + 60], d[jul1_idx + 335 : jul1_idx + 365]), axis=0)
            q3.append(tmp)
            q4.append(d[jul1_idx + 60 : jul1_idx + 152])
            q1.append(d[jul1_idx + 152 : jul1_idx + 244])
            q2.append(d[jul1_idx + 244 : jul1_idx + 335])

            jul1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
        mam_res = np.vstack(q4)
        jja_res = np.vstack(q1)
        son_res = np.vstack(q2)
        djf_res = np.vstack(q3)
    return mam_res, jja_res, son_res, djf_res
    
def seasonal_means(d):
    q = quarters(d, HIST_START, HIST_END)
    return np.array([np.mean(q[0], axis=1), np.mean(q[1], axis=1), np.mean(q[2], axis=1), np.mean(q[3], axis=1)])

def calibration_function(hist_obs, hist_mod):
# Calibration functions are P-P plots of historical and modeled values

    source = np.sort(hist_obs.flatten())
    target= np.sort(hist_mod.flatten())
   
    if (np.max(source) == 0 and np.min(source) == 0):
        return np.arange(0, target.size) / target.size
    if (np.max(target) == 0 and np.min(target) == 0):
        return np.arange(0, source.size) / source.size
    new_indices = []

    for target_idx, target_value in enumerate(target):
        if target_idx < len(source):
            source_value = source[target_idx]
            if source_value > target[-1]:
                new_indices.append(target.size - 1)
            else:
                new_indices.append(np.argmax(target >= source_value))
    return np.array(new_indices) / source.size

def calibrate_component(uncalibrated_data, calibration_fxn):
    N = len(uncalibrated_data)
    unsorted_uncalib = [(i, idx) for idx, i in enumerate(uncalibrated_data)]
    sorted_uncalib = sorted(unsorted_uncalib)
    result = [0] * N
    for j in range(N):
        X_j = j / (N + 1)
        Y_jprime = calibration_fxn[math.floor(X_j * len(calibration_fxn))]
        jprime = math.floor(Y_jprime * (N + 1))
        result[sorted_uncalib[j][1]] = sorted_uncalib[min(len(sorted_uncalib)-1, jprime)][0]
    
    return result

def calibrate(uncalibrated_data, calibration_fxn):
    mam = []
    jja = []
    son = []
    djf = []
    mam_idx = []
    jja_idx = []
    son_idx = []
    djf_idx = []
    for idx, i in enumerate(uncalibrated_data):
        if idx % 365 >= 60 and idx % 365 < 152:
            mam.append(uncalibrated_data[idx])
            mam_idx.append(idx)
        elif idx % 365 >= 152 and idx % 365 < 244:
            jja.append(uncalibrated_data[idx])
            jja_idx.append(idx)
        elif idx % 365 >= 244 and idx % 365 < 335:
            son.append(uncalibrated_data[idx])
            son_idx.append(idx)
        else:
            djf.append(uncalibrated_data[idx])
            djf_idx.append(idx)
    
    mam_calib = calibrate_component(np.array(mam), calibration_fxn[0])
    jja_calib = calibrate_component(np.array(jja), calibration_fxn[1])
    son_calib = calibrate_component(np.array(son), calibration_fxn[2])
    djf_calib = calibrate_component(np.array(djf), calibration_fxn[3])
    
    result = [0] * len(uncalibrated_data)
    for i in range(len(mam_idx)):
        result[mam_idx[i]] = mam_calib[i]
    for i in range(len(jja_idx)):
        result[jja_idx[i]] = jja_calib[i]
    for i in range(len(son_idx)):
        result[son_idx[i]] = son_calib[i]
    for i in range(len(djf_idx)):
        result[djf_idx[i]] = djf_calib[i]

    return np.array(result)

def get_gamma(count, size):
    return np.random.gamma(shape = count + 0.5, size=size)
def get_beta(count, num, size):
    return np.random.beta(a = count + 0.5, b = num - count + 0.5, size=size)

In [120]:
class Hazard:
    pass

class Tempwave_simple(Hazard):
    def __init__(self, varname, min_duration, threshold, want_gte=True):
        if type(threshold) == np.ndarray and threshold.size % 365 != 0:
            raise Exception('Comparison array length is not an integer multiple of 365')
        self.varname = varname
        self.want_gte = want_gte
        self.min_duration = min_duration
        self.threshold = threshold  # May be scalar or 365-long array
        self.probmodel = 'Poisson'
    def count(self, datalist):
        data = datalist[0]
        if type(self.threshold) in (float, int, np.float64, np.int32):
            threshold = self.threshold
        else:   # type is np array
            threshold = np.array([])
            while threshold.size < data.size:
                threshold = np.concatenate([threshold, self.threshold])
        if self.want_gte:
            tf_array = data >= threshold
        else:
            tf_array = data <= threshold
        return count_runs(tf_array, self.min_duration)

def wetbulbtemp(T, RH):
# JA Knox et al. 2017. Two simple and accurate approximations for wet-bulb
# temperature in moist conditions, with forecasting applications. Bull. Am.
# Meteorol. Soc. 98(9): 1897-1906. doi:10.1175/BAMS-D-16-0246.1
    T = T.astype(np.float64)
    rh_percent = RH.astype(np.float64) * 100
    return T * np.arctan(0.151977 * np.sqrt(rh_percent + 8.313659)) + np.arctan(T + rh_percent) - np.arctan(rh_percent - 1.676331) + ((0.00391838 * ((rh_percent)**(3/2))) * np.arctan(0.023101 * rh_percent)) - 4.686035


class WetbulbHeatwave(Hazard):
    def __init__(self, wbgt_threshold, min_duration):
        self.varname = 'tasmax+hurs'
        self.min_duration = min_duration
        self.wbgt_threshold = wbgt_threshold
        self.probmodel = 'Poisson'
    def count(self, datalist):
        data_t = datalist[0]
        data_h = datalist[1]
        data = wetbulbtemp(data_t, data_h)
        tf_array = data >= self.wbgt_threshold
        return count_runs(tf_array, self.min_duration)

class WetbulbDays(Hazard):
    def __init__(self, wbgt_threshold):
        self.varname = 'tasmax+hurs'
        self.wbgt_threshold = wbgt_threshold
        self.probmodel = 'binomial'
    def count(self, datalist):
        data_t = datalist[0]
        data_h = datalist[1]
        data = wetbulbtemp(data_t, data_h)
        byyear = data.reshape(data.size//365, 365)
        return np.sum((np.max(byyear, axis=1) >= self.wbgt_threshold) * 1)
    
class Heatwave_highlow(Hazard):
    def __init__(self, hightemp, lowtemp, min_duration):
        self.varname = 'tasmax+tasmin'
        self.min_duration = min_duration
        self.hightemp = hightemp
        self.lowtemp = lowtemp
        self.probmodel = 'Poisson'
    def count(self, datalist):
        data_tx = datalist[0]
        data_tn = datalist[1]
        if type(self.hightemp) in (float, int, np.float64, np.int32):
            high_threshold = self.hightemp
        else:   # type is np array
            high_threshold = np.array([])
            while high_threshold.size < data_tx.size:
                high_threshold = np.concatenate([high_threshold, self.hightemp])
        if type(self.lowtemp) in (float, int, np.float64, np.int32):
            low_threshold = self.lowtemp
        else:   # type is np array
            low_threshold = np.array([])
            while low_threshold.size < data_tn.size:
                low_threshold = np.concatenate([low_threshold, self.lowtemp])
        tf_array_tx = data_tx >= high_threshold
        tf_array_tn = data_tn >= low_threshold
        return count_runs(tf_array_tx * tf_array_tn, self.min_duration)

class Threshold_simple(Hazard):
    def __init__(self, varname, var_threshold, want_gte):
        self.varname = varname
        self.var_threshold = var_threshold
        self.want_gte = want_gte
        self.probmodel = 'binomial'
    def count(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size//365, 365)
        if self.want_gte:
            return np.sum(np.sum(byyear >= self.var_threshold, axis=1) >= self.count_threshold)
        else:
            return np.sum(np.sum(byyear <= self.var_threshold, axis=1) >= self.count_threshold)
    def count_nc(self, datalist, targetcount):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size//365, 365)
        return np.sum(np.abs(np.sum(byyear >= self.var_threshold, axis=1) - targetcount) < 0.5)
        

class Hotdays_inrange(Hazard):
    def __init__(self, hightemp, lowtemp):
        self.varname = 'tasmax'
        self.hightemp = hightemp
        self.lowtemp = lowtemp
        self.probmodel = 'binomial'
    def count(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        tf_array_high = data <= self.hightemp
        tf_array_low = data >= self.lowtemp
        return runs(tf_array_high * tf_array_low, self.min_duration, 'count')

In [114]:
a = np.array([2,3,-4,5])
np.abs(a)

array([2, 3, 4, 5])

In [125]:
class Location:
    def __init__(self, name, loc_id, latlon):
        self.name = name
        self.loc_id = loc_id
        self.latlon = latlon
        self.hist_observed = {}
        self.hist_modeled = {scenario: {} for scenario in FUTURE_SCENARIOS}
        self.best_models = {scenario: {} for scenario in FUTURE_SCENARIOS}
        self.fut_modeled = {scenario: {} for scenario in FUTURE_SCENARIOS}
        self.calib_fxns = {scenario: {} for scenario in FUTURE_SCENARIOS}
        
    def get_data(self, varname, future_start, future_end):
    # Gets historical observations from ERA5 Daily Aggregate (from GEE)
    # Goes through all NEX-GDDP-CMIP6 models in GEE and gets historical model outputs
    # Chooses three best models based on quarterly RMSD

        def relhum(T, Tdp):
            T = T.astype(np.float64)
            Tdp = Tdp.astype(np.float64)
            numerator = np.exp(17.625 * Tdp / (243.04 + Tdp))
            denominator = np.exp(17.625 * T / (243.04 + T))
            return 100 * numerator / denominator
        
        #print('  Getting historical {0}'.format(varname))
        if varname=='hurs':
            era_dewpoint = VARIABLES['tasmax']['era_transform'](get_eravar('dewpoint_2m_temperature', self.latlon, HIST_START, HIST_END, southern_hem=False))
            era_maxtemp = VARIABLES['tasmax']['era_transform'](get_eravar(VARIABLES['tasmax']['era_varname'], self.latlon, HIST_START, HIST_END, southern_hem=False))
            hist_obs = relhum(era_maxtemp, era_dewpoint)
        else:
            hist_obs = VARIABLES[varname]['era_transform'](get_eravar(VARIABLES[varname]['era_varname'], self.latlon, HIST_START, HIST_END, southern_hem=False))
        self.hist_observed[varname] = hist_obs
        for scenario in FUTURE_SCENARIOS:
            hist_mods = {}
            rmsds = []
            for model in MODELS[varname]:
                #print('    Getting {0}'.format(model))
                hist_mod = VARIABLES[varname]['nex_transform'](get_var(varname, model, self.loc_id, HIST_START, HIST_END, southern_hem=False, scenario=scenario))
                hist_mods[model] = hist_mod
                rmsds.append((get_rmsd(hist_obs, hist_mod), model))
            rmsds.sort()

            best_models = []
            families = []
            idx = 0
            while len(best_models) < NUM_BEST_MODELS:
                if not MODEL_FAMILY[rmsds[idx][1]] in families:
                    best_models.append(rmsds[idx][1])
                    families.append(MODEL_FAMILY[rmsds[idx][1]])
                idx += 1

            #for m in best_models:
            #    print(m, [i[0] for i in rmsds if i[1]==m][0])
            #best_models = []
            #for idx in range(min(NUM_BEST_MODELS, len(MODELS[varname]))):
            #    best_models.append(rmsds[idx][1])

            self.hist_modeled[scenario][varname] = hist_mods
            self.best_models[scenario][varname] = best_models

        # Get calibration functions
            #print('  Getting calibration functions')
            self.calib_fxns[scenario][varname] = {}
            hist_obs = self.hist_observed[varname]
            hist_mod = self.hist_modeled[scenario][varname]
            for model in self.best_models[scenario][varname]:
                o_quarters = quarters(hist_obs, HIST_START, HIST_END)
                m_quarters = quarters(hist_mod[model], HIST_START, HIST_END)
                self.calib_fxns[scenario][varname][model] = [calibration_function(o_quarters[i].flatten(), m_quarters[i].flatten()) for i in range(4)]

        # Get future model outputs
            #print('  Getting future {0}'.format(varname))
            self.fut_modeled[scenario][varname] = {
                model: VARIABLES[varname]['nex_transform'](get_var(varname, model, self.loc_id, future_start, future_end, extralong=True, scenario=scenario)) for model in best_models
            }
        
            

In [121]:
class Estimate:
    def __init__(self, location, hazard, future_start, future_end, sh_year):
        self.location = location
        self.hazard = hazard
        self.future_start = future_start
        self.future_end = future_end
        self.sh_year = sh_year 
        self.estimate = {scenario: {} for scenario in FUTURE_SCENARIOS}
        self.expected_estimate = {scenario: {} for scenario in FUTURE_SCENARIOS}
        
        for varname in self.hazard.varname.split('+'):
            if not varname in list(self.location.hist_observed.keys()):
                self.location.get_data(varname, future_start, future_end)
        
    def future_count(self):
    # Gets future model outputs for three best models
    # Calibrates based on stored model-specific quarterly calibration functions
    # Calculates event count within future time series
    # Draws 10,000 posterior rate parameters from appropriate Jeffrys prior distribution (parameterized with count)
    # For each rate parameter, draw one event count
        for scenario in FUTURE_SCENARIOS:
            fut_mod = {}
            varnames = self.hazard.varname.split('+')
            for varname in varnames:
                fut_mod[varname] = {}
                for model in self.location.best_models[scenario][varname]:
                    if self.sh_year:
                        fut_mod[varname][model] = self.location.fut_modeled[scenario][varname][model][:-184]
                    else:
                        fut_mod[varname][model] = self.location.fut_modeled[scenario][varname][model][184:]
            best_models = []
            for idx in range(min(len(MODELS[varname]), NUM_BEST_MODELS)):
                best_models.append('+'.join([self.location.best_models[scenario][varname][idx] for varname in varnames]))
            posterior_rateparams = {}
            posterior_draws = {}
            estimate = {}
            for modelplus in best_models:
                calib_data = []
                for idx, varname in enumerate(varnames):
                    model = modelplus.split('+')[idx]
                    calib_data.append(np.array(calibrate(fut_mod[varname][model], self.location.calib_fxns[scenario][varname][model])))
                if self.sh_year:
                    count = self.hazard.count(calib_data)
                else:
                    count = self.hazard.count([cd[152:-213] for cd in calib_data])
                if self.hazard.probmodel == 'Poisson':
                    posterior_rateparams[modelplus] = get_gamma(count, 10000)
                    posterior_draws[modelplus] = np.random.poisson(posterior_rateparams[modelplus], 10000)
                else:  # self.hazard.probmodel == 'binomial'
                    posterior_rateparams[modelplus] = get_beta(count, self.future_end - self.future_start + 1, 10000)
                    posterior_draws[modelplus] = np.random.binomial(self.future_end - self.future_start + 1, posterior_rateparams[modelplus], 10000)
                self.estimate[scenario][modelplus] = [np.percentile(posterior_draws[modelplus], q)/(self.future_end - self.future_start + 1) for q in (25, 50, 75)] #np.sum(posterior_draws[modelplus]) / 10000

            print()
            print(scenario)
            for modelplus in best_models:
                if self.hazard.probmodel == 'Poisson':
                    print('{0}: {1}'.format(modelplus, str([i for i in self.estimate[scenario][modelplus]])[1:-1]))
                else:
                    print('{0}: {1}'.format(modelplus, str(['{0:.1f}%'.format(i / (self.future_end - self.future_start + 1) * 100) for i in self.estimate[scenario][modelplus]])[1:-1]))
    
    def expected_count(self):
        for scenario in FUTURE_SCENARIOS:
            fut_mod = {}
            varnames = self.hazard.varname.split('+')
            for varname in varnames:
                fut_mod[varname] = {}
                for model in self.location.best_models[scenario][varname]:
                    if self.sh_year:
                        fut_mod[varname][model] = self.location.fut_modeled[scenario][varname][model][:-184]
                    else:
                        fut_mod[varname][model] = self.location.fut_modeled[scenario][varname][model][184:]
            best_models = []
            for idx in range(min(len(MODELS[varname]), NUM_BEST_MODELS)):
                best_models.append('+'.join([self.location.best_models[scenario][varname][idx] for varname in varnames]))
            estimates = {}
            for modelplus in best_models:
                res_sum = [0, 0, 0]
                for targetcount in range(0, 365):
                    
                    calib_data = []
                    for idx, varname in enumerate(varnames):
                        model = modelplus.split('+')[idx]
                        calib_data.append(np.array(calibrate(fut_mod[varname][model], self.location.calib_fxns[scenario][varname][model])))
                    if self.sh_year:
                        count = self.hazard.count_nc(calib_data, targetcount)
                    else:
                        count = self.hazard.count_nc([cd[152:-213] for cd in calib_data], targetcount)

                    posterior_rateparams = get_beta(count, self.future_end - self.future_start + 1, 10000)
                    posterior_draws = np.random.binomial(self.future_end - self.future_start + 1, posterior_rateparams, 10000)
                    
                    probs = [np.percentile(posterior_draws, q) / (self.future_end - self.future_start + 1) for q in (25, 50, 75)]
                    res_sum = [res_sum[idx] + (targetcount * probs[idx]) for idx in (0, 1, 2)]
                self.expected_estimate[scenario][modelplus] = [max(min(res_sum[idx], 365), 0) for idx in (0, 1, 2)]

                

In [13]:
%%time
datasets = {}
for varname in VARIABLES:
    for model in MODELS[varname]:
        for scenario in ['historical'] + FUTURE_SCENARIOS:
            datasets[(varname, model, scenario)] = Dataset(varname, model, scenario)

Extracting tasmax historical GFDL-ESM4
Extracting tasmax ssp119 GFDL-ESM4
Extracting tasmax ssp126 GFDL-ESM4
Extracting tasmax historical CanESM5
Extracting tasmax ssp119 CanESM5
Extracting tasmax ssp126 CanESM5
Extracting tasmax historical MRI-ESM2-0
Extracting tasmax ssp119 MRI-ESM2-0
Extracting tasmax ssp126 MRI-ESM2-0
Extracting tasmax historical IPSL-CM6A-LR
Extracting tasmax ssp119 IPSL-CM6A-LR
Extracting tasmax ssp126 IPSL-CM6A-LR
Extracting tasmax historical EC-Earth3-Veg-LR
Extracting tasmax ssp119 EC-Earth3-Veg-LR
Extracting tasmax ssp126 EC-Earth3-Veg-LR
CPU times: total: 13min 36s
Wall time: 32min 56s


In [ ]:
%%time
with open('test_outputs_2.csv', 'w') as ofile:
    ofile.write('Location,Scenario,Hazard,Model,Year_range,Q1,Q2,Q3,Tx_95y\n')
    for name in CITYLATLON:

        lat, lon, loc_id = CITYLATLON[name]
        print()
        print(loc_id)
        print(name)
        
        #high_95c = calendardate_percentiles('tasmax', 95, (lat, lon), sh_hem=lat < 0)
        Tx_95y = wholeyear_percentile('tasmax', 95, (lat, lon))
        median_95p_freq = thresholdexceedance_mediancount('tasmax', Tx_95y, (lat, lon), True)
        test_hazards = [
            {'name': 'Heat wave 95th percentile', 'obj': Tempwave_simple('tasmax', 5, Tx_95y, True), 'sh_year': lat < 0},
            {'name': 'Days warmer than {0:.2f}C'.format(Tx_95y), 'obj': Threshold_simple('tasmax', Tx_95y, want_gte=True), 'sh_year': lat < 0}
        ]
        print('  95th pctl maxtemp = {0:.2f}'.format(Tx_95y))
        print('  median num hist days >= {0} = {1}'.format(Tx_95y, median_95p_freq))
       # print('  5th pctl mintemp = {0:.1f}'.format(cold_5n))
        loc = Location(name, loc_id, (lat, lon))
    
        for haz in test_hazards:
            
            for fut_start, fut_end in FUTURE_YEARS:
                print()
                print(haz['name'])
                print(fut_start, fut_end)
                est = Estimate(loc, haz['obj'], fut_start, fut_end, haz['sh_year'])
                if haz['obj'].probmodel == 'Poisson':
                    est.future_count()
                    for scenario in FUTURE_SCENARIOS:
                        for model in est.estimate[scenario]:
                            res = list(est.estimate[scenario][model])
                            ofile.write('{0},{1},{2},{3},{4},{5},{6},{7},{8}\n'.format(loc.name, haz['name'], scenario, model, '{0}-{1}'.format(fut_start, fut_end), res[0], res[1], res[2], Tx_95y))

                else:
                    est.expected_count()
                    for scenario in FUTURE_SCENARIOS:
                        for model in est.expected_estimate[scenario]:
                            res = list(est.expected_estimate[scenario][model])
                            print('{0} {1}: {2}'.format(scenario, model, ','.join([str(i) for i in res])))
                            ofile.write('{0},{1},{2},{3},{4},{5},{6},{7},{8}\n'.format(loc.name, haz['name'], scenario, model, '{0}-{1}'.format(fut_start, fut_end), res[0], res[1], res[2], Tx_95y))
        del(loc)
        del(est)


0
Dubai AE
  95th pctl maxtemp = 40.21
  median num hist days >= 40.210791015625 = 17.0

Heat wave 95th percentile
2080 2099

ssp119
MRI-ESM2-0: 2.2, 2.5, 2.85
GFDL-ESM4: 1.5, 1.8, 2.1
IPSL-CM6A-LR: 2.05, 2.35, 2.7

ssp126
MRI-ESM2-0: 2.15, 2.5, 2.85
GFDL-ESM4: 2.1, 2.4, 2.75
IPSL-CM6A-LR: 1.9, 2.2, 2.5

Days warmer than 40.21C
2080 2099
ssp119 MRI-ESM2-0: 6.300000000000001,35.05,365
ssp119 GFDL-ESM4: 9.05,24.65,365
ssp119 IPSL-CM6A-LR: 4.45,28.6,365
ssp126 MRI-ESM2-0: 10.3,38.25,365
ssp126 GFDL-ESM4: 3.5,32.800000000000004,365
ssp126 IPSL-CM6A-LR: 5.800000000000001,30.45,365

1
Lezha AL


In [34]:
for quarter in range(4):
    obs_10 = np.percentile(quarters(hist_obs_tx, HIST_START, HIST_END)[quarter], 10)
    obs_90 = np.percentile(quarters(hist_obs_tx, HIST_START, HIST_END)[quarter], 90)
    for model in best_models_tx:
        mod = quarters(hist_mods_tx[model] - 273.15, HIST_START, HIST_END)[quarter].flatten()
        print('{0}: min modeled value does not exceed observed 10th percentile  {1}'.format(model, min(mod) <= obs_10))
        print('{0}: max modeled value does not exceed observed 90th percentile  {1}'.format(model, max(mod) >= obs_90))


GFDL-CM4: min modeled value does not exceed observed 10th percentile  True
GFDL-CM4: max modeled value does not exceed observed 90th percentile  True
CanESM5: min modeled value does not exceed observed 10th percentile  True
CanESM5: max modeled value does not exceed observed 90th percentile  True
ACCESS-CM2: min modeled value does not exceed observed 10th percentile  True
ACCESS-CM2: max modeled value does not exceed observed 90th percentile  True
GFDL-CM4: min modeled value does not exceed observed 10th percentile  True
GFDL-CM4: max modeled value does not exceed observed 90th percentile  True
CanESM5: min modeled value does not exceed observed 10th percentile  True
CanESM5: max modeled value does not exceed observed 90th percentile  True
ACCESS-CM2: min modeled value does not exceed observed 10th percentile  True
ACCESS-CM2: max modeled value does not exceed observed 90th percentile  True
GFDL-CM4: min modeled value does not exceed observed 10th percentile  True
GFDL-CM4: max modeled